In [1]:
import pandas as pd


In [2]:
df=pd.read_excel('clean_data_.xlsx')

In [3]:
df

,project_name,spec,floor,area,value
0,Pride Purple Park Grandeur,4 BHK Apartment,Higherof 25 floors,3000 sq.ft,4.5 Cr
1,Rahul Arcus Bldg E,4 BHK Apartment,Middleof 18 floors,2858 sq.ft,2.52 Cr
2,Paranjape Yuthika,2 BHK Apartment,Higherof 12 floors,1150 sq.ft,1.5 Cr
3,Pride Purple Park Grandeur,4 BHK Apartment,Higherof 25 floors,3000 sq.ft,4.5 Cr
4,Rahul Arcus Bldg E,4 BHK Apartment,Middleof 18 floors,2858 sq.ft,2.52 Cr
...,...,...,...,...,...
1856,Supreme Estia,3 BHK Apartment,Middleof 22 floors,1620 sq.ft,1.5 Cr
1857,Aditya Comfort Zone,2 BHK Apartment,Middleof 13 floors,1161 sq.ft,89.0 L
1858,Shilpa Rucha Homes,1 BHK Apartment,Lowerof 10 floors,675 sq.ft,55.0 L
1859,Supreme Estia,3 BHK Apartment,Middleof 19 floors,982 sq.ft,1.11 Cr


In [4]:
df.spec.value_counts()

3 BHK Apartment                    867
2 BHK Apartment                    542
4 BHK Apartment                    352
5 BHK Apartment                     60
1 BHK Apartment                     20
4 BHK Independent Builder Floor     11
1 RK Apartment                       9
Name: spec, dtype: int64

### remove the 1 Rk Apartment from data from spec

In [5]:

df1=df[(df.spec!='1 RK Apartment') & (df.spec!='4 BHK Independent Builder Floor')]

In [6]:
df1.head()

,project_name,spec,floor,area,value
0,Pride Purple Park Grandeur,4 BHK Apartment,Higherof 25 floors,3000 sq.ft,4.5 Cr
1,Rahul Arcus Bldg E,4 BHK Apartment,Middleof 18 floors,2858 sq.ft,2.52 Cr
2,Paranjape Yuthika,2 BHK Apartment,Higherof 12 floors,1150 sq.ft,1.5 Cr
3,Pride Purple Park Grandeur,4 BHK Apartment,Higherof 25 floors,3000 sq.ft,4.5 Cr
4,Rahul Arcus Bldg E,4 BHK Apartment,Middleof 18 floors,2858 sq.ft,2.52 Cr


### to use machine learning on this data we have to process this data, remove BHK Apartment from spec, Higher lower, middle and extra text from floor, sq.ft from area and change lakh into crore and then remove the cr and lakh

In [7]:
# remove the BHK Apartment from df
import re
df2=df1.copy()
df2.spec=df2.spec.apply(lambda x: re.findall('\d{1,3}',x)[0]).values

In [8]:
# convert floor and area
df2.floor=df2.floor.apply(lambda x:re.findall('\d{1,3}',x)[0]).values
df2.area=df2.area.apply(lambda x: x.replace(' sq.ft','')).values

In [9]:
df2

,project_name,spec,floor,area,value
0,Pride Purple Park Grandeur,4,25,3000,4.5 Cr
1,Rahul Arcus Bldg E,4,18,2858,2.52 Cr
2,Paranjape Yuthika,2,12,1150,1.5 Cr
3,Pride Purple Park Grandeur,4,25,3000,4.5 Cr
4,Rahul Arcus Bldg E,4,18,2858,2.52 Cr
...,...,...,...,...,...
1856,Supreme Estia,3,22,1620,1.5 Cr
1857,Aditya Comfort Zone,2,13,1161,89.0 L
1858,Shilpa Rucha Homes,1,10,675,55.0 L
1859,Supreme Estia,3,19,982,1.11 Cr


In [10]:
def convert_lakh_crore(col):
    if re.findall('L',col):
        val=col.replace(' L','')
        val=float(val)
        cr=val/100
        return cr
    else:
        val=col.replace(' Cr','')
        val=float(val)
        return val

In [11]:
df3=df2.copy()

In [12]:
# convert value
df3.value=df3.value.apply(convert_lakh_crore).values

In [13]:
df3.dtypes

project_name     object
spec             object
floor            object
area             object
value           float64
dtype: object

In [14]:
# we have to change the dtype becuase here dtype of spec , floor and area is object
df3.area=df3.area.apply(lambda x: int(x)).values
df3.floor=df3.floor.apply(lambda x: int(x)).values
df3.spec=df3.spec.apply(lambda x: int(x)).values

In [15]:
df3.dtypes

project_name     object
spec              int64
floor             int64
area              int64
value           float64
dtype: object

In [16]:
len(df3.project_name.unique())

87

In [17]:
# we have to convert the categorical column project_name into numerical to apply ml models

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
lb=LabelEncoder()


In [20]:
df4=df3.copy()

In [21]:
df4.project_name=lb.fit_transform(df4.project_name)

In [22]:
df4

,project_name,spec,floor,area,value
0,51,4,25,3000,4.50
1,56,4,18,2858,2.52
2,43,2,12,1150,1.50
3,51,4,25,3000,4.50
4,56,4,18,2858,2.52
...,...,...,...,...,...
1856,77,3,22,1620,1.50
1857,3,2,13,1161,0.89
1858,69,1,10,675,0.55
1859,77,3,19,982,1.11


# Now our data is ready for training

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [24]:
x=df4.drop('value',axis=1)
y=df4['value']

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [26]:
lr=LinearRegression()
dr=DecisionTreeRegressor()
rf=RandomForestRegressor()
# svr=SVR(kernel='linear',gamma='auto')
svr=SVR()


### linear regression

In [27]:
lr.fit(x_train,y_train)

LinearRegression()

In [28]:
lr.score(x_test,y_test)

0.8413051002720572

### decision tree

In [29]:
dr.fit(x_train,y_train)

DecisionTreeRegressor()

In [30]:
dr.score(x_test,y_test)

0.9982300637890952

### random forest

In [31]:
rf.fit(x_train,y_train)

RandomForestRegressor()

In [32]:
rf.score(x_test,y_test)

0.9980311732694452

### SVM

In [35]:
svr.fit(x_train,y_train)


SVR()

In [36]:
svr.score(x_test,y_test)

0.8162198444990156

##### save the clean data without duplicates

In [38]:
df5=df.drop_duplicates()

In [41]:
df5.to_csv('final_data_without_duplicates.csv',index=None)
